This notebook runs `post_process_visit_lists` to take the outputs of `getDC2VisitList` and produce the per-band visit lists.

First, we run the code on Run 1 visits, since the per-band visits lists for Run 1.1 were produced by Rahul's code (https://github.com/LSSTDESC/DC2-production/blob/master/Notebooks/survey_design_visit_lists.ipynb). We compare the lists produced here wit the other code's outputs (sitting here: https://github.com/LSSTDESC/DC2-production/tree/master/data/Run1.1) to ensure things are working.
- WFD lists match. uDDF ones don't since we are not implementing a selection criterion based on night and airmass.

Then, we run the code to produce the per-band lists for Run 2.

In [1]:
import numpy as np
import pandas as pd
import os
import time

In [2]:
os.chdir('/global/u2/a/awan/LSST/lsstRepos/DC2_visitList/')
!python setup.py install --user

running install
running build
running build_py
running install_lib
running install_egg_info
Removing /global/homes/a/awan/.local/lib/python3.6/site-packages/dc2visitgen-0.1-py3.6.egg-info
Writing /global/homes/a/awan/.local/lib/python3.6/site-packages/dc2visitgen-0.1-py3.6.egg-info


In [3]:
from dc2visitgen import post_process_visit_lists

In [4]:
help(post_process_visit_lists)

Help on function post_process_visit_lists in module dc2visitgen.post_process_visitList:

post_process_visit_lists(dir_to_readfrom, wfd_filename, uddf_filename, base_filetag, outDir, saveData=True, nVisits=None, dbpath='/global/cscratch1/sd/awan/dbs_old_unzipped/minion_1016_desc_dithered_v3.db')
    The goal here is to take the all-band visit lists produced by DC2_visitListCode, separate them by band
    and save the obsHistIDs and expMJD corresponding to all the visits in the given band.
    
    Essentially following https://github.com/LSSTDESC/DC2-production/blob/master/Notebooks/survey_design_visit_lists.ipynb
    
    HOWEVER, Run 1.1 uDDF per-band production code include grouping by night, out of which the visits with the minimum airmass
    were chosen. Since all the visits need to be simulated for Run 2 and not just a subset, the uDDF lists are created the same
    way as for WFD: the visits are only grouped by band and ordered by expMJD.



In [5]:
mainDir = '/global/homes/a/awan/LSST/lsstRepos/DC2_visitList/DC2visitGen/'

#### First check the lists that would be compareable to the ones for Run 1.1 produced by Rahul's notebook

In [6]:
# create the per-band lists
nside = 512
baseDir = '%s/protoDC2Visits_nside%s/'%(mainDir, nside)
out = post_process_visit_lists(dir_to_readfrom=baseDir,
                               wfd_filename='DC2VisitList_minion1016_v3_WFDvisits_nside%s_WFDvisitsInWFD.csv'%nside,
                               uddf_filename='DC2VisitList_minion1016_v3_DDvisits_nside%s_DDvisitsInuDD.csv'%nside,
                               base_filetag='check',
                               saveData=True,
                               outDir='%s/visitLists/per_band_lists'%(baseDir),
                               nVisits={'WFD':1125, 'uDDF':879*19 + 170}  # from Rahul's notebook
                        )

Working with lsst.sims.maf.__version__: 2.9.0.sims

## Working with WFD

All-band array: shape = (8389, 2)

Working with g-band
91 visits for g-band.

Saved check_g-band_WFDvisits.txt
in /global/homes/a/awan/LSST/lsstRepos/DC2_visitList/DC2visitGen//protoDC2Visits_nside512//visitLists/per_band_lists.

Working with i-band
223 visits for i-band.

Saved check_i-band_WFDvisits.txt
in /global/homes/a/awan/LSST/lsstRepos/DC2_visitList/DC2visitGen//protoDC2Visits_nside512//visitLists/per_band_lists.

Working with r-band
245 visits for r-band.

Saved check_r-band_WFDvisits.txt
in /global/homes/a/awan/LSST/lsstRepos/DC2_visitList/DC2visitGen//protoDC2Visits_nside512//visitLists/per_band_lists.

Working with u-band
67 visits for u-band.

Saved check_u-band_WFDvisits.txt
in /global/homes/a/awan/LSST/lsstRepos/DC2_visitList/DC2visitGen//protoDC2Visits_nside512//visitLists/per_band_lists.

Working with y-band
252 visits for y-band.

Saved check_y-band_WFDvisits.txt
in /global/homes/a/awan/LSST/lsst

We already see that uDDF visits wont match since the numVisit for uDDF give many more visits here than they did earlier.

In [7]:
# now read in the data 
outDir_old = '/global/homes/a/awan/LSST/lsstRepos/DC2-production/data/Run1.1/'
outDir_new = '%s/protoDC2Visits_nside%s/visitLists/per_band_lists/'%(mainDir, nside)

In [8]:
for survey in out.keys():
    print('---------------------------------------------------------------------------------------------------')
    print('### Reading in %s files\n'%survey)
    for band in ['u', 'g', 'r', 'i', 'z', 'y']:
        print('## Reading in %s-band files\n'%band)

        old_files = [f for f in os.listdir(outDir_old) if f.__contains__('%s-band'%band) and f.__contains__(survey)]
        print('Found %s old files'%len(old_files))
        
        new_files = [f for f in os.listdir(outDir_new) if f.__contains__('%s-band'%band) and f.__contains__(survey)]
        print('Found %s new files'%len(new_files))
        
        if len(old_files)>1:
            raise ValueError('Somethings wrong: %s'%old_files)
        if len(new_files)>1:
            raise ValueError('Somethings wrong: %s'%new_files)
        
        print('Reading in %s'%old_files[0])
        old_data = pd.read_csv('%s/%s'%(outDir_old, old_files[0]))
        print('Reading in %s\n'%new_files[0])
        new_data = pd.read_csv('%s/%s'%(outDir_new, new_files[0]), skiprows=1)  # need to skip the first row since have a header
        
        
        if np.shape(old_data)==np.shape(new_data):
            print('###### Old, new match: %s\n'%(np.unique(old_data==new_data)))
        else:
            print('###### Old, new DO NOT match in size: old vs. new: %s vs %s\n'%(np.shape(old_data), np.shape(new_data)))        

---------------------------------------------------------------------------------------------------
### Reading in WFD files

## Reading in u-band files

Found 1 old files
Found 1 new files
Reading in protoDC2_visits_WFD_u-band.txt
Reading in check_u-band_WFDvisits.txt

###### Old, new match: [ True]

## Reading in g-band files

Found 1 old files
Found 1 new files
Reading in protoDC2_visits_WFD_g-band.txt
Reading in check_g-band_WFDvisits.txt

###### Old, new match: [ True]

## Reading in r-band files

Found 1 old files
Found 1 new files
Reading in protoDC2_visits_WFD_r-band.txt
Reading in check_r-band_WFDvisits.txt

###### Old, new match: [ True]

## Reading in i-band files

Found 1 old files
Found 1 new files
Reading in protoDC2_visits_WFD_i-band.txt
Reading in check_i-band_WFDvisits.txt

###### Old, new match: [ True]

## Reading in z-band files

Found 1 old files
Found 1 new files
Reading in protoDC2_visits_WFD_z-band.txt
Reading in check_z-band_WFDvisits.txt

###### Old, new match

- WFD lists match -- good. 
- Run 1.1 uDDF per-band production code include grouping by night, out of which the visits with minimum airmass were chosen -- not something that is implemented in the code for Run 2, so do not expect the lists to match.

#### Now create the per-band lists for Run 2.

In [9]:
# create the per-band lists
nside = 1024
baseDir = '%s/DC2_Run2_Visits_nside%s/'%(mainDir, nside)
out = post_process_visit_lists(dir_to_readfrom=baseDir,
                               wfd_filename='DC2VisitList_minion1016_v3_WFDvisits_nside%s_WFDvisitsInWFD.csv'%nside,
                               uddf_filename='DC2VisitList_minion1016_v3_DDvisits_nside%s_DDvisitsInuDD.csv'%nside,
                               base_filetag='DC2_Run2',
                               saveData=True,
                               outDir='%s/visitLists/per_band_lists'%(baseDir),
                               nVisits=None
                        )

Working with lsst.sims.maf.__version__: 2.9.0.sims

## Working with WFD

All-band array: shape = (50415, 2)

Working with g-band
4873 visits for g-band.

Saved DC2_Run2_g-band_WFDvisits.txt
in /global/homes/a/awan/LSST/lsstRepos/DC2_visitList/DC2visitGen//DC2_Run2_Visits_nside1024//visitLists/per_band_lists.

Working with i-band
11133 visits for i-band.

Saved DC2_Run2_i-band_WFDvisits.txt
in /global/homes/a/awan/LSST/lsstRepos/DC2_visitList/DC2visitGen//DC2_Run2_Visits_nside1024//visitLists/per_band_lists.

Working with r-band
11096 visits for r-band.

Saved DC2_Run2_r-band_WFDvisits.txt
in /global/homes/a/awan/LSST/lsstRepos/DC2_visitList/DC2visitGen//DC2_Run2_Visits_nside1024//visitLists/per_band_lists.

Working with u-band
3468 visits for u-band.

Saved DC2_Run2_u-band_WFDvisits.txt
in /global/homes/a/awan/LSST/lsstRepos/DC2_visitList/DC2visitGen//DC2_Run2_Visits_nside1024//visitLists/per_band_lists.

Working with y-band
9934 visits for y-band.

Saved DC2_Run2_y-band_WFDvisits.txt
